In [21]:
import glob
import gzip
from bs4 import BeautifulSoup
import lxml
from collections import defaultdict
from tqdm import tqdm
import requests
import random
import sys
import pathlib
import csv



import argparse

# import multiprocessing
from fuzzywuzzy import fuzz

# url = 'http://ai-capo-api-lb/spaCy_ner_predictor?text_sentence='  # Load Balancer

result_path = '/mnt/droplet/nfs/gns/literature/machine-learning/evaluation/FP_Analysis/SpaCy/model_call/'
pathlib.Path(result_path).mkdir(parents=True, exist_ok=True)



In [7]:
import spacy
from spacy import util
best_model_path = '/mnt/droplet/nfs/gns/literature/Santosh_Tirunagari/GitHub/spacy_models/pretrain_exp/best/'

print("Loading from", best_model_path)
nlp2 = util.load_model_from_path(best_model_path)

Loading from /mnt/droplet/nfs/gns/literature/Santosh_Tirunagari/GitHub/spacy_models/pretrain_exp/best/


In [22]:
import pandas as pd

path_EPMC_text = '/mnt/droplet/nfs/gns/literature/machine-learning/evaluation/300articles/europePMC-NER/annotations_API/full_sentences/test_annotations/Europe_PMC_annotation.csv'
colNames = ['pmc_id', 'section', 'sentence','ner'] 
    
test_df = pd.read_csv(path_EPMC_text,sep ='\t', names=colNames) 

In [23]:
from ast import literal_eval

def deleting_epmc_GPS(list_1,del_name):
   
    for sub_list in list_1:
        if del_name in sub_list:
            list_1.remove(sub_list)
    return list_1

In [24]:
from fuzzywuzzy import fuzz

def remove_FP(epmc_list, ml_json):
    all_ml_gps = []
    if ml_json['annotations']:
        for each_ml_annotation in ml_json['annotations']:
            if each_ml_annotation[2] == 'GP':
                all_ml_gps.append(each_ml_annotation[3])
    
    non_FP_removed =[]
              

    for each_ner in epmc_list:
        if each_ner[2] == 'Gene_Proteins':
            for each_ml_gp in all_ml_gps:         
                score = fuzz.token_set_ratio(each_ml_gp, each_ner[1])
                if score == 100:
                    non_FP_removed.append(each_ner)

    non_gp_tags =  deleting_epmc_GPS(epmc_list,'Gene_Proteins')  

    fp_removed_tags = non_gp_tags+non_FP_removed

    return fp_removed_tags


In [25]:
def get_spacy_annotations(text_sentence):
    data_dict ={}
    doc = nlp2(text_sentence)
    terms_entities = []
    
    for ent in doc.ents:
        terms_entities.append(
            [ent.start_char, ent.end_char, ent.label_, ent.text])
    
    data_dict['annotations'] = terms_entities
    
    return data_dict

In [26]:
# from tqdm import tqdm
with open(result_path + 'spacy_fp_removal_80.tsv', 'w', newline='\n') as f1:
    public_writer = csv.writer(f1, delimiter='\t', lineterminator='\n')
    
    for index,row in tqdm(test_df.iterrows(),total = len(test_df)):
        try:
            ml_annotations = get_spacy_annotations(row['sentence'])
            fp_removed = remove_FP(literal_eval(row['ner']), ml_annotations)
        except ValueError:
            fp_removed =''
            
        public_writer.writerow([row['pmc_id'], row['section'],row['sentence'], fp_removed])   
            
    

100%|██████████| 17957/17957 [00:48<00:00, 368.81it/s]


In [27]:
# Convert to IOB format

import sys
import os
import glob

sys.path.append('/mnt/droplet/nfs/gns/literature/Santosh_Tirunagari/test Gitlab/epmc-ml-misc-library/')

import capo_tools_lib
import evaluation_epmc_lib




In [30]:
iob_result_path = result_path+'iob/'
pathlib.Path(iob_result_path).mkdir(parents=True, exist_ok=True)

file_path = result_path + 'spacy_fp_removal_80.tsv'
capo_tools_lib.annotations_api_tagged_sentences_to_IOB(file_path,
                                                       iob_result_path,'spacy_fp_removal_iob.tsv')

100%|██████████| 17957/17957 [00:03<00:00, 5244.33it/s]


In [31]:
import metrics.ner as ner_metrics


# precision	0.7	0.7	0.72	0.73
# recall	0.53	0.53	0.54	0.55
# f1 score	0.6	0.6	0.62	0.6

#
# print(ner_metrics.semeval_scores_report(gold=epmc_labels, response=ml_labels, digits=2))

root_path = '/mnt/droplet/nfs/gns/literature/machine-learning/'
epmc_path = root_path+'Datasets/NER_Datasets/EBI_standard-IOB/test.csv'
all_tags = ['GP', 'DS', 'OG']

print('################ Annotation Pipeline Results ########################')
CAPO_path = iob_result_path+'spacy_fp_removal_iob.tsv'
for each_tag in all_tags:
    print('############ '+each_tag+' ####################')
    print('\n')
    print(ner_metrics.semeval_report(gold_path=epmc_path, response_path=CAPO_path, targets=[each_tag]))

################ Annotation Pipeline Results ########################
############ GP ####################


               strict     exact   partial      type

   correct      2,986     2,986     2,986     3,129
 incorrect        143       143         0         0
   partial          0         0       143         0
   missing      3,440     3,440     3,440     3,440
  spurious        530       530       530       530
 precision       0.82      0.82      0.84      0.86
    recall       0.45      0.45      0.47      0.48
  f1 score       0.58      0.58      0.60      0.61
Gold Total      6,566
Resp Total      3,659


############ DS ####################


               strict     exact   partial      type

   correct      1,095     1,095     1,095     1,215
 incorrect        120       120         0         0
   partial          0         0       120         0
   missing      1,135     1,135     1,135     1,135
  spurious        449       449       449       449
 precision       0.66   